### Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [236]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [242]:
keywords = ['python', 'парсинг']
url = 'https://habr.com/ru/all/'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

In [241]:
blocks = soup.find_all('article', class_='post post_preview')
blocks
news = []
for preview in blocks:
    preview_lower = preview.text.lower()
    for key in keywords:
        if key in preview_lower:
            
            date_element = preview.find('span', class_ = 'post__time')
            title_element = preview.find('a', class_='post__title_link')
            link_element = title_element.attrs.get('href')
           
            news.append({'date': date_element.text, 'title': title_element.text, 'link': link_element})
            
data = pd.DataFrame(news)
data


,date,title,link
0,сегодня в 18:52,Развертывание интерактивных визуализаций данны...,https://habr.com/ru/company/skillfactory/blog/...
1,сегодня в 18:34,25 лучших репозиториев GitHub для разработчико...,https://habr.com/ru/post/526334/


### Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [247]:
import json
import datetime
import time
time.sleep(0.2)
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
params = {
    "emailAddresses":['xxxx@gmail.ru', 'yyyy@rambler.ru'],
}
headers = {
    'Vaar-Version':'0',
    'Vaar-Header-App-Product':'hackcheck-web-avast'
}
res = requests.post(url, headers=headers, data=json.dumps(params))
res.json()

hack_email = {}
data = pd.DataFrame()

for email, hack in res.json()['summary'].items():
    hack_email[email] = list(hack.values())[0]

for id_, details in res.json()['breaches'].items():
    for email, hack in hack_email.items():
        if int(id_) in hack:
            row = {'Почта': email, 
                   'Дата утечки': details['publishDate'], 
                   'Источник утечки': details['site'],
                   'Описание утечки': details['description']
                  }
            data = pd.concat([data, pd.DataFrame([row])])

data

,Почта,Дата утечки,Источник утечки,Описание утечки
0,xxxx@gmail.ru,2019-03-07T00:00:00Z,dubsmash.com,"In December 2018, Dubsmash's database was alle..."
0,yyyy@rambler.ru,2016-10-21T00:00:00Z,myspace.com,"Shortly before the 2016 Memorial Day weekend, ..."
0,yyyy@rambler.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,xxxx@gmail.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,yyyy@rambler.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
0,yyyy@rambler.ru,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
0,yyyy@rambler.ru,2016-10-23T00:00:00Z,rambler.ru,"On February 17, 2012, Russian email provider R..."
0,yyyy@rambler.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,yyyy@rambler.ru,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
